<a href="https://colab.research.google.com/github/Amrutha4561/labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

3.The choice of the percentage to reserve for the validation set depends on several factors, including the size of your dataset, the complexity of your model, and the nature of your data. There is no one-size-fits-all answer, but common splits that strike a balance between having a sufficiently large training set and a reasonably representative validation set often fall in the range of 20% to 30% for the validation set. This means that 70% to 80% of the data is typically used for training.

Here are some considerations to help you decide on an appropriate percentage for the validation set:

1. **Dataset Size**: If you have a large dataset, you can afford to allocate a smaller percentage to the validation set, as you still have a substantial amount of data for training. Conversely, with a smaller dataset, you might want to allocate a larger percentage to validation to make reliable performance estimates.

2. **Complexity of the Model**: More complex models often require larger training datasets to capture their intricacies. If you're using a very complex model, consider reserving a larger portion for training and a smaller portion for validation.

3. **Data Variability**: If your data is highly variable and diverse, you may need a larger validation set to ensure it represents the entire data distribution adequately.

4. **Cross-Validation**: If you have limited data, techniques like cross-validation can help you make the most of the available data. Cross-validation involves splitting the data into multiple folds, using each as a validation set in turn. This can provide a more robust estimate of your model's performance.

5. **Experimentation**: It's often a good practice to experiment with different validation set sizes to see how they affect model performance. You can try various splits and measure how they impact your model's ability to generalize to unseen data (i.e., the test set).

In practice, starting with a 70-80% training and 20-30% validation split is a reasonable default. However, be prepared to adjust this based on the specific characteristics of your data and problem. The key is to strike a balance that allows your model to train effectively and also provides a reliable estimate of its performance on unseen data.

2.The size of the training and validation sets can affect how well you can predict the accuracy on the test set using the validation set. Here's how:

1. **Larger Training Set:**
   - When you have a larger training set, your model has more data to learn from, which can lead to better generalization. With more diverse and representative training data, your model is likely to capture the underlying patterns in the data more effectively.

   - This can result in a model that performs better on the validation set. If your model generalizes well on the validation set, it's more likely to generalize well on the test set too. So, a larger training set often provides a more optimistic estimate of how well your model will perform on unseen data (the test set).

2. **Larger Validation Set:**
   - A larger validation set can provide a more reliable estimate of your model's performance, as it's evaluated on a more substantial portion of the data. This can help you make a more accurate prediction of how well your model will perform on the test set.

   - However, a larger validation set may leave you with a smaller training set, potentially leading to underfitting if the training data isn't sufficient to capture the complexity of the problem.

3. **Trade-off and Balance:**
   - There's often a trade-off between the size of the training and validation sets. You want a sufficiently large validation set to make a reliable performance estimate, but you also need a sufficiently large training set to train a model that generalizes well.

   - Striking the right balance is essential. You typically see common splits like 70-80% for training and 20-30% for validation. These splits are often chosen because they provide a reasonable balance between training and validation data.

4. **Generalization Prediction:**
   - The goal of using a validation set is to predict how well your model will generalize to unseen data, represented by the test set. If your validation set is too small or not representative of the data distribution, it may not accurately predict the model's performance on the test set.

In summary, the size of the training and validation sets can impact your ability to predict how well your model will perform on the test set. A larger training set can lead to better generalization, but a larger validation set can provide a more reliable estimate of performance. Finding the right balance between these two sets is crucial to make accurate predictions about your model's performance on unseen data.

The accuracy of a machine learning model on the validation set can be affected by the size of the validation set. Let's explore how it is affected when you increase or decrease the percentage of the validation set:

1. **Increasing the Percentage of the Validation Set:**
   - When you increase the percentage of the validation set, you provide the model with more data for validation. This can have several effects:
   - Pros:
     - The model may have a better estimate of its performance because it's evaluated on a larger, more representative portion of the data.
     - It can help identify overfitting more effectively, as the model is validated on a larger independent dataset.
   - Cons:
     - You have less data for training, which can potentially lead to underfitting, where the model doesn't learn the underlying patterns in the data well.
     - If your dataset is small to begin with, allocating too much data to the validation set may leave an insufficient amount for training.

2. **Reducing the Percentage of the Validation Set:**
   - When you reduce the percentage of the validation set, you provide the model with less data for validation. This can also have several effects:
   - Pros:
     - You have more data available for training, which can help the model learn better and potentially perform better on the training data.
   - Cons:
     - The model's estimate of its performance on the validation set may be less reliable because it's evaluated on a smaller and potentially less representative portion of the data.
     - It may be harder to detect overfitting as the validation set is smaller and may not effectively capture the model's generalization performance.

In practice, the ideal size of the validation set depends on various factors, including the size of your dataset, the complexity of your model, and the nature of your data. It's often a trade-off between having a sufficient validation set to make reliable performance estimates and having enough data for training.

Common practices include splitting the data into 70-80% for training and 20-30% for validation. However, you should experiment with different validation set sizes and use techniques like cross-validation to find the best setup for your specific problem. In general, having a reasonably sized validation set is crucial for effectively assessing and improving your model's performance.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


4.Increasing the number of iterations in cross-validation can help when dealing with a very small training dataset or validation dataset to some extent, but it may not fully overcome the limitations of having a small amount of data. While increasing the iterations can provide more robust estimates and make better use of the available data, it has its limitations:

**Advantages of Increasing Iterations:**
1. **Improved Stability**: More iterations can lead to more stable and reliable estimates of model performance, even when the training and validation datasets are small. It helps reduce the impact of randomness and data variability.

2. **Better Hyperparameter Tuning**: With limited data, it may be challenging to perform thorough hyperparameter tuning. Increasing iterations allows you to assess the model's performance under different settings, which can be valuable for optimizing hyperparameters.

3. **Maximizing Data Utilization**: If you have a very small dataset, using a high number of iterations ensures that you make the most of the limited data available, as the data is divided into multiple subsets for training and validation in each iteration.

**Limitations of Increasing Iterations:**
1. **Data Size Remains Limited**: Cross-validation can't magically create more data; it just maximizes the utility of the available data. If the training dataset is extremely small, increasing the number of iterations can only help to a limited extent. The model's ability to generalize effectively may still be constrained by the overall lack of training data.

2. **Computation Overhead**: While increasing iterations is beneficial for assessment, it comes with a computational cost. Training and evaluating the model multiple times can be time-consuming and resource-intensive, particularly if your dataset is small to begin with.

3. **Risk of Overfitting the Validation Set**: In cases of extremely small validation sets, increasing iterations may risk overfitting the validation data because there's little data available for validation. This can lead to an overly optimistic estimate of model performance.

In situations where you have very small training or validation datasets, it's crucial to be cautious about the trade-offs. Consider the following approaches:

1. **Collect More Data**: If possible, acquiring more data is often the best solution to address the limitations of small datasets.

2. **Regularization**: Use strong regularization techniques to prevent overfitting when dealing with small training sets.

3. **Simple Models**: Consider using simpler models that require fewer parameters and are less prone to overfitting when data is limited.

4. **Feature Engineering**: Focus on feature engineering to extract more information from the limited data available.

5. **Careful Model Evaluation**: Be aware that model performance estimates may still be limited by the data size, and results should be interpreted with caution.

In summary, while increasing the number of iterations in cross-validation can improve estimates, it is not a substitute for having an adequate amount of data. Dealing with very small datasets remains a challenging problem, and various strategies beyond cross-validation may be needed to address the limitations imposed by data scarcity.

3.The number of iterations, or folds, in cross-validation can have an impact on the estimate of a model's performance. In general, using a higher number of iterations can lead to a more robust and reliable estimate, but there are trade-offs to consider.

Here are the effects of the number of iterations on the estimate in cross-validation:

1. **More Iterations (Higher K in k-fold Cross-Validation):**
   - **Pros:**
     - A higher number of iterations provides a more comprehensive assessment of your model's performance. It allows you to evaluate the model's generalization across a wider range of validation subsets, reducing the potential impact of data variability.
     - You obtain a more stable and reliable estimate of model performance, as the results are averaged over a larger number of folds.
   - **Cons:**
     - The computational cost increases with more iterations, as the model needs to be trained and evaluated more times. This can be a concern with large datasets or computationally expensive models.
     - There might be diminishing returns in terms of improved estimate accuracy. After a certain point, increasing the number of iterations may not significantly enhance the estimate.

2. **Fewer Iterations (Lower K in k-fold Cross-Validation):**
   - **Pros:**
     - Fewer iterations are computationally less expensive, which can be a consideration for large datasets or complex models.
     - It may be sufficient for assessing model performance, especially if the data is homogeneous and you have limited computational resources.
   - **Cons:**
     - A lower number of iterations can result in more variable estimates, as the model's performance is assessed on a smaller number of validation sets. The estimate may be less reliable due to the limited diversity in the validation subsets.

In practice, the choice of the number of iterations in cross-validation (e.g., the value of "K" in k-fold cross-validation) is often a trade-off between computational resources and the desire for a more accurate and robust estimate. Common choices for K in k-fold cross-validation include 5, 10, or 20, but the optimal value can vary depending on the specific dataset and problem.

It's important to balance the need for a reliable estimate with practical constraints. If computational resources are not a limiting factor, using a higher number of iterations is generally a good practice. However, be mindful of diminishing returns and consider your specific goals and constraints when selecting the appropriate number of iterations for cross-validation.

2.Cross-validation provides a more accurate estimate of the test accuracy than using a single validation set in cases where you do not have a separate, held-out test set. However, it's important to clarify the terminology to avoid any confusion:

1. **Test Set**: The "test set" is a completely independent dataset that is not used during model development, training, or validation. It is held out until the end and used to assess how well your model will generalize to unseen data. This is the most accurate estimate of a model's performance.

2. **Validation Set**: The "validation set" is used to tune model hyperparameters and assess model performance during training. It's typically a portion of the dataset that is not used in the initial model training but is used to make decisions about model selection, hyperparameter tuning, and to estimate the model's performance.

3. **Cross-Validation**: Cross-validation, like k-fold cross-validation or leave-one-out cross-validation, involves splitting your data into multiple training and validation sets and repeatedly assessing model performance on different subsets of your data. It provides a more reliable and robust estimate of how well your model will generalize to unseen data compared to using a single validation set.

Cross-validation helps you estimate the model's performance more accurately than a single validation set because it leverages different subsets of your data for validation, providing a more comprehensive evaluation of your model's ability to generalize. However, it's still an estimate and not as accurate as an independent test set.

To obtain the most accurate estimate of test accuracy, it's advisable to have a separate, completely independent test set that is not used in the training, validation, or hyperparameter tuning process. This test set provides the most reliable assessment of your model's performance on unseen data. Cross-validation is valuable when you don't have a separate test set or want to make the best use of your data for model assessment, but it's still an estimate and not a direct replacement for a true test set.

1.Yes, averaging the validation accuracy across multiple splits in cross-validation can give more consistent and robust results compared to a single validation set. Cross-validation techniques like k-fold cross-validation and leave-one-out (LOO) are designed to provide a more reliable estimate of a model's performance by systematically cycling through different subsets of the data. Here's why averaging the validation accuracy in cross-validation is beneficial:

1. **Reduced Variance**: Averaging over multiple splits helps reduce the impact of randomness and data variability. If you have a single validation set, its performance can be influenced by the particular data points in that set. By performing multiple splits, you get a more stable estimate because the impact of individual data points is diluted across the folds.

2. **Better Generalization Assessment**: Cross-validation allows you to assess how well your model generalizes to different subsets of the data. It provides a more comprehensive evaluation of your model's performance, which is important for assessing its reliability and robustness.

3. **Reduced Bias**: Averaging across multiple splits helps mitigate potential bias introduced by a single split. If a single split happens to be unrepresentative of the overall data distribution, it can lead to biased estimates of model performance. Cross-validation minimizes this risk.

4. **Optimal Model Selection**: When you use cross-validation for hyperparameter tuning or model selection, averaging the results from different splits provides a more accurate measure of the best-performing model. This is essential for making informed choices regarding model complexity, regularization parameters, and other hyperparameters.

5. **More Information**: Cross-validation provides additional insights, such as variance in performance across different data subsets, which can be valuable for assessing the robustness and stability of your model.

Common variants of cross-validation, such as k-fold cross-validation and leave-one-out (LOO), divide the data into multiple subsets and cycle through them, reporting the average performance over all the iterations. This process gives a more comprehensive and reliable assessment of a model's accuracy.

In summary, averaging the validation accuracy across multiple splits in cross-validation is a standard practice in machine learning because it provides more consistent and informative results, reduces variance and bias, and helps ensure that your model's performance estimates are more reliable and generalizable.